In [1]:
import pandas as pd
cb = pd.read_csv('CoffeeBean.csv', encoding='cp949',index_col=0,header= 0,engine='python')

In [2]:
def change_city(city):
  city = city.strip()
  if city == '서울시' or city == '서울':
    return '서울특별시'
  elif city == '경남':
    return '경상남도'
  elif city == '충북':
    return '충청북도'
  elif city == '충남':
    return '충청남도'
  elif city == '제주도':
    return '제주특별자치도'
  elif city == '경기':
    return '경기도'
  elif city == '강원':
    return '강원특별자치도'
  elif city == '부산시' or city == '부산':
    return '부산광역시'
  elif city == '대전시':
    return '대전광역시'
  elif city == '광주시':
    return '광주광역시'
  elif city == '대구시':
    return '대구광역시'
  elif city == '울산시':
    return '울산광역시'
  elif city == '세종시':
    return '세종특별자치시'
  elif city == '인천시':
    return '인천광역시'
  elif city == '광주':
    return '광주광역시'
  else:
    return city

In [3]:
addr = []
for address in cb['address']:
  temp = address.split()
  temp[0] = change_city(temp[0])
  addr.append(' '.join(temp) )

cb['address']  = addr
cb.head()

store                              address        phone
0   차병원점            서울특별시 강남구 논현로 566 강남차병원1층  02-538-7615
1  강남대로점                서울특별시 서초구 강남대로 369 1층  02-588-5778
2  청담에스점  서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호  02-548-6052
3    신사점                   서울특별시 강남구 도산대로 126  02-548-2741
4    역삼점             서울특별시 강남구 논현로 512 지상1,2층  02-569-8051

In [4]:
import requests
def get_geocoding(address):
  try:
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    REST_API_KEY = '485d4b5f6d8324bc7f21958d160d1860'
    header = {
        'Authorization': f'KakaoAK {REST_API_KEY}'
    }
    params = {
        'query': address
    }
    response = requests.get(url, headers=header, params=params)
    result = response.json()
    x = result['documents'][0]['road_address']['x']
    y = result['documents'][0]['road_address']['y']
    return x,y
  except:
    return None, None

In [9]:
cb['x'] = None
cb['y'] = None

In [18]:
for index in cb.index:
  cb.loc[index,'x'],cb.loc[index,'y'] = get_geocoding(cb.loc[index,'address'])

In [19]:
cb.to_csv('CoffeeBean_geo.csv',encoding='cp949',index=False)

In [26]:
cb.head(10)


store                               address         phone  \
0     차병원점             서울특별시 강남구 논현로 566 강남차병원1층   02-538-7615   
1    강남대로점                 서울특별시 서초구 강남대로 369 1층   02-588-5778   
2    청담에스점   서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호   02-548-6052   
3      신사점                    서울특별시 강남구 도산대로 126   02-548-2741   
4      역삼점              서울특별시 강남구 논현로 512 지상1,2층   02-569-8051   
5  양재스포타임점           서울특별시 서초구 강남대로 213 24호 지하1층   02-578-6833   
6    청담성당점           서울특별시 강남구 삼성로 716 LEE76빌딩2층   02-542-2053   
7      영동점           서울특별시 서초구 반포동 736-17 P빌딩 2층  02-3443-2096   
8      도곡점  서울특별시 강남구 언주로 30길 10,112 현대비젼21 112호   02-572-2781   
9    영동고앞점                서울특별시 강남구 선릉로 749 1,2층   02-544-3794   

                  x                 y  
0  127.034714418284  37.5070324421316  
1  127.028208465188  37.4956734817567  
2  127.047232410937  37.5252547717564  
3  127.022409708358  37.5169658359413  
4  127.036521514294   37.502349282282  
5  127.034949554107  37.4827666361943  
6  127.050036555762  37.5199060766075  
7   127.02281904918  37.5068586518937  
8              None              None  
9              None              None

In [36]:
# y, x
import folium
map_cb = folium.Map(location=[37.56,126.97],zoom_start=11)  # 지도의 중심좌표와 배율 을 설정
for idx, rowdata in cb.iterrows(): # 순번, 각 row 데이터
  if rowdata['x'] == None or rowdata['y'] == None:
    continue
  folium.Marker([rowdata['y'],rowdata['x']],popup=rowdata['store']).add_to(map_cb)

In [37]:
map_cb